# Ant Colony Optimization based Itinerary Recommender

## "The title is pretty much self-explanatory. We will be following the ACO framework shown below:"

###### ACO Framework 

% Create Graph

% Draw Graph

%% ACO algorithm

    % Define Initial Parameters
    
    % Main loop
    
    for i = 1 to max_iter:
        % Create ants
        % Calculate fitness value of all ants
        % Find the best ant 
        % Update the pheromone matrix
        % Implement Evaporation
        % Display results 
    end

In [ ]:
# Import the libraries 

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Import the dataset
top_dests_ktm=pd.read_csv("../../Datasets/destinations_of_kathmandu_updated_with_latlong.csv")

In [ ]:
print(top_dests_ktm.head())

In [ ]:
# select the first 95 destinations as they're the destinations having the latitude and longitude currently
top_dests_ktm=top_dests_ktm[:95]

In [ ]:
print(top_dests_ktm.tail())

In [ ]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests_ktm[['title','latitude','longitude']])

In [ ]:
# # let's plot the graph again 
# %matplotlib inline 
# plt.figure(figsize=(12,12)) 

# for lat,long in zip(top_dests_ktm['latitude'],top_dests_ktm['longitude']):
    
#     for another_lat,another_long in zip(top_dests_ktm['latitude'],top_dests_ktm['longitude']):
        
#         # create a list holding the two latitude
#         lat_list=[lat,another_lat]
#         # also create a list holding the two longitudes 
#         long_list=[long,another_long]
#         plt.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
# #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')

# for title,x,y in zip(top_dests_ktm['title'],top_dests_ktm['latitude'],top_dests_ktm['longitude']):

#     label = "{}".format(title)

#     plt.annotate(label, # this is the text
#                  (x,y), # these are the coordinates to position the label
#                  textcoords="offset points", # how to position the text
#                  xytext=(0,10), # distance from text to points (x,y)
#                  ha='center') # horizontal alignment can be left, right or center
# plt.show()

In [ ]:
# let's just pick n=10 for now for better visualization
n=95 #graph size
top_n_dests=top_dests_ktm[:n]
print(top_n_dests)

# define a starting point 
starting_point=0

In [ ]:
# remove the duplicate entry

print(len(top_n_dests['latitude'].unique()))
print(len(top_n_dests['longitude'].unique()))

#print out the duplicated latitude,longitude
duplicate_lat_long = top_n_dests[top_n_dests.duplicated(['latitude','longitude'])]
print(duplicate_lat_long)

print(top_n_dests[top_n_dests['latitude']=='27.7154575'])

In [ ]:
def plot_graph(ax,fig):
    """
        Function to plot the graph of matrix for every edge between the given nodes
    """

    for lat,long in zip(top_n_dests['latitude'],top_n_dests['longitude']):
        for another_lat,another_long in zip(top_n_dests['latitude'],top_n_dests['longitude']):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]

            ax.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            fig.canvas.draw()
            
    #adding label to each node
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):

        label = "{}".format(title)

        ax.annotate(label, # this is the text
                     (x,y), # these are the coordinates to position the label
                     textcoords="offset points", # how to position the text
                     xytext=(-5,5), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or center



In [ ]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n,n))

def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1

        #increment i
        i+=1

In [ ]:
create_graph()
print(graph)

In [ ]:
# Define the initial parameters of ACO
max_iter=100 #max no of iterations
ant_no=10 #number of ants 

#initial pheromone concentration
tau_0=10*1/(n*np.mean(graph))

#initial pheromone matrix
tau = tau_0*np.ones((n,n))
print("Initial pheromone matrix (Tau):")
print(tau)

#desirability/quality of each edge,eta
eta=1/graph #inverse of cost (distance)
print("Desirability matrix (Eta):")
print(eta)

rho=0.05 #evaporation rate
alpha=2 #pheromone exponential parameter
beta=1 #edge desirabiltu/quality exponential parameter


In [ ]:
print(tau[2,:]**alpha)

In [ ]:
print(eta[2,:]**beta)

## Now moving on to the main loop of ACO

In [ ]:
# defining a function to create the ant colony 
def create_colony(graph,node_no,ant_no,tau,eta,alpha,beta,initial_node):
    """
        creates an ant colony
    """
    # create a list called 'colony'
    # inside it, there will be ants ( represented by the indices)
    # each position contains other list called 'tour' that contains
    # the tour completed by each ant
    colony=[[] for _ in range(ant_no)] #size of the colony=no of ants
    
    # Run iterations equal to the number of ants 
    for i in range(ant_no):
        
        # define the initial node
        # we should take it as a parameter actually 
        # but let's take it random for now
        # initial_node=np.random.randint(0,node_no)
        
        # nope we changed our mind 
        initial_node=initial_node
        colony[i].append(initial_node)
        
        print(colony)
        
        for j in range(node_no-1):
            #to choose the rest of the nodes
            
            #current node is the last node in the list
            current_node=int(colony[i][-1])
#             print("current node:",current_node)
            
            #use the following formula to calculate the probability 
            p_all_nodes=(tau[current_node,:]**alpha)*(eta[current_node,:]**beta)
#             print("Prob:",p_all_nodes)
            
            #make the probability of the visited nodes 0
#             print("colony:",colony)
            for node in colony[i]:
                p_all_nodes[node]=0
                
            #calculate the final probability
            p=p_all_nodes/np.sum(p_all_nodes)
#             print("Sum prob:",np.sum(p_all_nodes))
#             print("final prob:",p)
            
            #Note: 'p'is a vector having size (1,node_no)   
#             print("probability vector:",p)
            
            #select the next node based on roulette wheel
            next_node=roulette_wheel(p)
            
            #append next node to the tour 
            #made by the ant in the colony
            colony[i].append(next_node)
            
        # to complete the tour, 
        # add the initial node to the end as well
        
        colony[i].append(initial_node)
    return colony

In [ ]:
# let's implement the roulette wheel function
def roulette_wheel(prob_vec):
    """
        Selects the next node based on 
        the cumulative sum of the probabilities 
        in the probability vector
    """
    #cumulative version of the prob_vec (p)
    
    cum_sum_p=np.cumsum(prob_vec)
    
    #choose a random value from 0 to 1
    random_val=np.random.rand()
#     print("random value:",random_val)
    
    #choose the first index as next_node in the probability vector
    #that has value more or equal to the random_val
    
#     print("cum sum:",cum_sum_p)
    
    next_node = np.argwhere(cum_sum_p>=random_val)[0][0]
    
    #return the next_node
    return next_node
    

### Now. let's define a function that calculates the fitness value of the route calculated

In [ ]:
def fitness_function(tour,graph):
    """
        calculates the fitness of the tour
        > basically, it's the sum of cost of all edges
        in the tour.
    """
    fitness=0
    
    for i in range(len(tour)-1):
        
        current_node=tour[i] # current node
        next_node=tour[i+1] # next node
        
        # add the cost of current edge (current_node->next_node)
        # to the overall fitness
        
        fitness = fitness + graph[current_node][next_node]
        
    return fitness

### And a function to draw Pheromone graph

In [ ]:
def draw_pheromone(ax,fig,tau,graph):
    """
        Draws pheromone graph
    """
    ax.clear()

    # Maximum value of pheromone
    max_tau=np.max(tau)
    # Minimum value of pheromone
    min_tau=np.min(tau)
    
    # Normalize the tau matrix
    tau_normalized = (tau - min_tau)/(max_tau - min_tau)
    for lat,long,i in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):
        for another_lat,another_long,j in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]
            ax.plot(lat_list,long_list,color=(0,0,1-tau_normalized[i][j]), marker='o', linestyle='solid',linewidth=10*tau_normalized[i][j]+0.5, markersize=8,mfc = 'white')
            fig.canvas.draw()
            fig.canvas.flush_events()

    
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
    
            

### Also, a function to draw the best tour 

In [ ]:
def draw_best_tour(ax,fig,best_tour,graph):
    """
        function to draw the best tour
    """
    ax.clear()

    for i in range(len(best_tour)-1):
        #extract the current node and the next node
        current_node=best_tour[i]
        next_node=best_tour[i+1]
        
        #draw a line between current node and next node
        
        #latitude and longitude of the current node
        lat_current,long_current=top_n_dests.iloc[current_node]['latitude'],top_n_dests.iloc[current_node]['longitude']
        
        #latitude and longitude of the next node
        lat_next,long_next=top_n_dests.iloc[next_node]['latitude'],top_n_dests.iloc[next_node]['longitude']
        
        lat_list=[lat_current,lat_next]
        long_list=[long_current,long_next]
        
        plt.plot(lat_list,long_list,color='green', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
        fig.canvas.draw()
        fig.canvas.flush_events()

        
    #for annotation
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

### Also , a function to update the pheromone matrix

In [ ]:
def update_pheromone(tau,colony,fitness_list):
    """
        function to update the pheromone matrix
    """
    node_no=len(colony[1])
    ant_no=len(colony)
    
    for i in range(ant_no):
        # for each ant
        for j in range(node_no-1):
            # for each node in the tour 
            current_node=colony[i][j]
            next_node=colony[i][j+1]
            
            tau[current_node,next_node]=tau[current_node,next_node]+1/fitness_list[i]
            tau[next_node,current_node]=tau[next_node,current_node]+1/fitness_list[i]
    
    return tau

## Main ACO loop 

In [ ]:
# Main loop

best_fitness=np.inf
best_tour=[]

# %matplotlib notebook

# fig = plt.figure(figsize=(10,10))
# ax1 = fig.add_subplot(131)
# ax2 = fig.add_subplot(132)
# ax3 = fig.add_subplot(133)

# ax1.title.set_text('Initial Mesh')
# ax2.title.set_text('Pheromone Graph Plot')
# ax3.title.set_text('Best Tour Plot')
# plt.ion()

# # plt.title("Pheromone graph plot:")
# fig.show()
# fig.canvas.draw()

#let's draw the initial graph (mesh) first
# plot_graph(ax1,fig)

for i in range(max_iter):
    # create ant colony
    colony=[] # store it as a list
    colony=create_colony(graph,n,ant_no,tau,eta,alpha,beta,starting_point)
    print(f"Iteration #{i}:")
#     print(colony)
    
    # initializing fitness_list
    fitness_list=[0]*ant_no
    
    # calculate the fitness value of all ants
    for ant_i in range(ant_no):
        fitness_list[ant_i]=fitness_function(colony[ant_i],graph)
        
#     print(fitness_list)
    
    # find the best ant 
    min_value=np.min(fitness_list)
    min_index=np.argmin(fitness_list) #best ant
    
    if min_value<best_fitness:
        # replace best_fitness even smaller min_value is found
        best_fitness=min_value
        best_tour=colony[min_index] #tour of the best ant
        
    print("Best fitness: ",best_fitness)
    print("Best tour: ",best_tour)
    
    # update phermone matrix
    tau=update_pheromone(tau,colony,fitness_list)
    
    # print("Updated pheromone matrix:",tau)
    
    # apply evaporation
    tau=(1-rho)*tau
    
    # print("Updated pheromone matrix:",tau)
    
    # plot the pheromone graph
    # draw_pheromone(ax2,fig,tau,graph)
    
    # plot the best tour
    # draw_best_tour(ax3,fig,best_tour,graph)

#     plt.subplot(1,3,1)
#     %matplotlib tk
#     plt.figure(1)
    
#     draw_pheromone(tau,graph)

    
    
# drawing the best tour plot 
# draw_best_tour(ax3,fig,best_tour,graph)

# drawing end pheromone graph
# draw_pheromone(ax2,fig,tau,graph)

# display the names of destinations in the last tour
print("Best tour obtained:")
print(best_tour)
for i in best_tour:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

In [ ]:
# plt.savefig('sample.png')